In [ ]:
from IPython.core.display import display, Markdown, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
import datetime
import tqdm
import importlib

In [ ]:
import xarray as xr
import numpy as np
%matplotlib notebook
%matplotlib notebook
import matplotlib.pyplot as plt

In [ ]:
from databroker import catalog

In [ ]:
import bact_analysis
import bact_analysis.bba.calc
import bact_analysis.transverse.calc
import bact_analysis.transverse.distorted_orbit
import bact_analysis.transverse.process
import bact_analysis.utils.preprocess
import bact_analysis_bessyii.bba.preprocess_data

# BBA Analysis

This notebook should be the center for selecting the required data and presenting the results.
All computation or data rearangement shall be performed in libraries

In [ ]:
datetime.datetime.now().strftime('%Y %m %d %H:%M')

## Data selection, check and load

In [ ]:
catalog_name = 'heavy'

In [ ]:
db = catalog[catalog_name]

For measurement elog entry see http://elog-v2.trs.bessy.de:8080/Machine+Devel.,+Comm./1788

In [ ]:
# uid = '24ff319d-b260-46e6-8356-0dfb08638fb1'
uid = '24ff319d-b260-46e6-8356-0dfb08638fb1'
uid = 'c6c3ad04-7c4c-4a6d-a7e8-91602cfea726'

In [ ]:
list(db.keys());

In [ ]:
run = db[uid]

In [ ]:
type(run)

In [ ]:
descriptor, = run.primary.metadata['descriptors']
descriptor;

configuration = descriptor['configuration']
dt_configuration = configuration['dt']
list(configuration.keys()), list(dt_configuration.keys())

### loading data

Loading dask seems to be the fastest approach

start = datetime.datetime.now()
all_data_ = run.primary.to_dask()
end = datetime.datetime.now()

end - start

Now load all data

In [ ]:
for name, item in tqdm.tqdm(all_data_.items(), total=len(all_data_.variables)):
    item.load()

### load lattice model

In [ ]:
selected_model = xr.load_dataset('bessii_twiss_tracy.nc')
selected_model

### Checking consistency between lattice model and digital twin ... for the required elements

In [ ]:
device_name = 'dt'
descriptor, = run.primary.metadata['descriptors']
configuration = descriptor['configuration']
dt_configuration = configuration[device_name]

In [ ]:
importlib.reload(bact_analysis.utils.preprocess)

In [ ]:
if False:
    bpm_names = dt_configuration['data']['dt_bpm_waveform_names']
    bpm_names_lc = [name.lower() for name in bpm_names]
else:
    bpm_names = all_data_.dt_bpm_waveform_names.isel(time=0).values
    bpm_names_lc = [name.lower() for name in bpm_names]

In [ ]:
bpm_names_check = set(bpm_names_lc)

Check that all bpm's are in the lattice

In [ ]:
bpm_names_check.difference(selected_model.coords["pos"].values)

Remove first reading ... bpm data not garanteed to be correct

In [ ]:
idx = all_data_.dt_cs_setpoint != 0#
all_data__ = all_data_.isel(time=idx)

In [ ]:
all_data__.dt_cs_setpoint.max();

### Collapsing the number of different dimensions

Replace them with known lablled alternatives

In [ ]:
bpm_dims = bact_analysis_bessyii.bba.preprocess_data.replaceable_dims_bpm(
    all_data__, prefix="dt_", expected_length=len(bpm_names)
)

These are only relevant for digital twin data. Steerer data are not further processed here thus these are ignored.

### Derive info on measurement

When were magnets strength was switched or magnet was reselected

In [ ]:
muxer_pc_current_change = bact_analysis.utils.preprocess.enumerate_changed_value(all_data_.dt_mux_power_converter_setpoint)
muxer_pc_current_change.name = "muxer_pc_current_change" 
muxer_or_pc_current_change = bact_analysis.utils.preprocess.enumerate_changed_value_pairs(all_data_.dt_mux_power_converter_setpoint, all_data_.dt_mux_selector_selected)
muxer_or_pc_current_change.name = "muxer_or_pc_current_change" 

### Combine all info to new xarray 

In [ ]:
replace_dims = {dim : 'bpm' for dim in bpm_dims}
# replace_dims.update({dim : 'pos' for dim in beam_dims})
all_data = all_data_.rename(replace_dims).assign_coords(bpm=list(bpm_names), pos=selected_model.coords['pos'])
all_data

In [ ]:
preprocessed = xr.merge([all_data, muxer_pc_current_change, muxer_or_pc_current_change])
preprocessed

In [ ]:
type(dt_configuration)

In [ ]:
importlib.reload(bact_analysis_bessyii.bba.preprocess_data)

In [ ]:
preprocessed, dt_configuration = bact_analysis_bessyii.bba.preprocess_data.load_and_check_data(run)

### Make data selectable per magnet

In [ ]:
importlib.reload(bact_analysis.utils.preprocess)
importlib.reload(bact_analysis.transverse.calc)
importlib.reload(bact_analysis_bessyii.bba.preprocess_data)

In [ ]:
type(preprocessed)

In [ ]:
rearranged = xr.concat(
    bact_analysis.utils.preprocess.reorder_by_groups(
        preprocessed,
        preprocessed.groupby(preprocessed.dt_mux_selector_selected),
        reordered_dim="name",
        dim_sel="time",
        new_indices_dim="step",
    ),
    dim="name",
)

In [ ]:
rearranged;

In [ ]:
measurement_vars = dict(
    dt_bpm_waveform_x_pos="x_pos",
    dt_bpm_waveform_y_pos="y_pos",
    dt_mux_power_converter_setpoint="excitation",
)
redm4proc = (
    rearranged[list(measurement_vars.keys())]
    .rename_vars(**measurement_vars)
    .sel(bpm=bpm_names)
    .rename_dims(bpm="pos")
    .assign_coords(pos=bpm_names_lc)
)
# BPM Data are in mm
redm4proc['x_pos'] =  redm4proc.x_pos/ 1000
redm4proc['y_pos'] =  redm4proc.y_pos/ 1000

## Processing data

### Using model information

This model information is obtained here from the model produced by the digital shadow. 

Should be replaced by a standard lattice from database

## Test processing capabilities

In [ ]:
# d = {name: item for name, item in red4proc.coords.items()}

In [ ]:
#red4proc.x_pos.expand_dims?

In [ ]:
importlib.reload( bact_analysis.transverse.distorted_orbit)
importlib.reload( bact_analysis.transverse.calc)
importlib.reload( bact_analysis.transverse.process)

In [ ]:
start = datetime.datetime.now()
result = {
    name: item for name, item in 
    tqdm.tqdm(bact_analysis.transverse.process.process_all_gen(selected_model, redm4proc, redm4proc.coords['name'].values, bpm_names=bpm_names_lc, theta=1e-5), 
              total=len(redm4proc.coords['name']))
}
end  = datetime.datetime.now()
end - start

In [ ]:
rds = bact_analysis.transverse.process.combine_all(result)

In [ ]:
rds.orbit.attrs

## Check calculated orbits

In [ ]:
redm4proc.sel(name='Q1M1D1R')

In [ ]:
def check_kick_fit(measurement, orbit, parameters):
    """
    
    Todo:
          include measurement error
    """
    bpm_names = measurement.coords['pos']
    
    s = parameters.sel(parameter='scaled_angle')
    scale = s.sel(result='value')
    scale_err = s.sel(result='error')
    del s
    
    pars  = parameters.sel(parameter=bpm_names)
    offset =  pars.sel(result='value')
    offset_err = pars.sel(result='error')
    del pars
    
    #print(parameters)
    scaled_orbit = orbit * scale
    scaled_orbit_err = np.absolute(orbit) * scale_err
    
    scaled_orbit_at_bpm = scaled_orbit.sel(pos=bpm_names)
    s_x = scaled_orbit_at_bpm.sel(plane="x") * measurement.excitation
    s_y = scaled_orbit_at_bpm.sel(plane="y") * measurement.excitation
    m_x  = offset.sel(plane="x") - measurement.x_pos
    m_y  = offset.sel(plane="y") - measurement.y_pos
    diff_x = s_x + m_x
    diff_y = s_y + m_y
    del scaled_orbit_at_bpm
    
    so_at_bpm_err = scaled_orbit_err.sel(pos=bpm_names)
    diff_x_err = so_at_bpm_err.sel(plane="x") * measurement.excitation + offset_err.sel(plane="x")
    diff_y_err = so_at_bpm_err.sel(plane="y") * measurement.excitation + offset_err.sel(plane="y")
    del so_at_bpm_err
    
    pos, step = diff_x.dims
    coords = [["x", "y"], ["value", "error", "orbit", "measurement"]]
    dims = ["plane", "result",  pos, step]
    coords += [diff_x.coords[pos], diff_x.coords[step]]
    diff = xr.DataArray(data=[[diff_x, diff_x_err, s_x, m_x], [diff_y, diff_y_err, s_y, m_y]], dims=dims, coords=coords)
    return diff

In [ ]:
def process(name): 
    measurement = redm4proc.sel(name=name)
    orbit = rds.orbit.sel(name=name)
    diff = check_kick_fit(measurement, orbit, rds.fit_params.sel(name=name))
    diff = diff.expand_dims(name=[name])
    return diff

diffs_fit = xr.concat([process(name) for name in rds.coords["name"].values], dim="name")


In [ ]:
np.absolute(diffs_fit.sel(result="value")).max() * 1e6, np.absolute(diffs_fit.sel(result="error")).max() * 1e6

In [ ]:
rng = np.random.default_rng()

In [ ]:
quad_names = [str(name.values) for name in rds.coords["name"]]
quad_names_lc = [name.lower() for name in quad_names]

In [ ]:
y_max =  rds.fit_params.sel(plane="y", result="value", parameter="scaled_angle").argmax().values
y_min =  rds.fit_params.sel(plane="y", result="value", parameter="scaled_angle").argmin().values
x_max =  rds.fit_params.sel(plane="x", result="value", parameter="scaled_angle").argmax().values
x_min =  rds.fit_params.sel(plane="x", result="value", parameter="scaled_angle").argmin().values
y_max, y_min

In [ ]:
redm4proc.excitation.min()

In [ ]:
name = rng.choice(quad_names)
name = rds.coords["name"][91]
name = rds.coords["name"][x_min]
name = "Q4M2D1R"
name = "Q3M2T2R"
name = "Q4M1T4R"
name = "Q5M2T5R"
t_diff = diffs_fit.sel(name=name)

fig, axes = plt.subplots(2, 2, figsize=[14, 6], sharex=True)
ax_comp, ax_diff = axes
ax_x, ax_y = ax_comp
ax_dx, ax_dy = ax_diff
a_scale = 1e6 / 2
err_scale = a_scale * 1
ds = selected_model.ds.sel(pos=t_diff.pos)

for step in t_diff.step:
    t_dI = redm4proc.excitation.sel(name=name, step=step)
    a_scale_ = a_scale * np.sign(t_dI)
    err_scale_ = err_scale #* np.sign(step)
    ax_x.errorbar(
        ds, t_diff.sel(step=step, result="orbit", plane="x") * a_scale_,
        yerr=t_diff.sel(step=step, result="error", plane="x")* err_scale_, 
        fmt= '-'
    )
    ax_y.errorbar(
        ds, t_diff.sel(step=step, result="orbit", plane="y") * a_scale_,
        yerr=t_diff.sel(step=step, result="error", plane="y")* err_scale_, 
        fmt= '-'
    )
    ax_x.errorbar(
        ds, t_diff.sel(step=step, result="measurement", plane="x") * a_scale_ * -1,
        yerr=t_diff.sel(step=step, result="error", plane="x")* err_scale_, 
        fmt= '+'
    )
    ax_y.errorbar(
        ds, t_diff.sel(step=step, result="measurement", plane="y") * a_scale_ * -1,
        # yerr=t_diff.sel(step=step, result="error", plane="y") err_scale_, 
        fmt= '+'
    )
    ax_dx.errorbar(
        ds, t_diff.sel(step=step, result="value", plane="x") * a_scale_,
        yerr=t_diff.sel(step=step, result="error", plane="x")* err_scale_, 
        fmt= '.'
    )
    ax_dy.errorbar(
        ds, t_diff.sel(step=step, result="value", plane="y") * a_scale_,
        yerr=t_diff.sel(step=step, result="error", plane="y")* err_scale_, 
        fmt= '.'
    )
ax_y.set_xlabel('s [m]')
ax_x.set_ylabel('x [$\mu rad$]')
ax_y.set_ylabel('y [$\mu$ rad]');
ax_dx.set_ylabel('dx [$\mu rad$]')
ax_dy.set_ylabel('dy [$\mu$ rad]');


In [ ]:
fig, axes = plt.subplots(2, 1, figsize=[12, 9], sharex=True)
ax_x, ax_y = axes
a_scale = rds.orbit.attrs['theta'] * 1e6
err_scale = a_scale * 5
ds = selected_model.ds.sel(pos=quad_names_lc)
ax_x.errorbar(
    ds, rds.fit_params.sel(name=quad_names, parameter='scaled_angle', result='value', plane='x') * a_scale, 
    yerr=rds.fit_params.sel(name=quad_names, parameter='scaled_angle', result='error', plane='x') * err_scale, 
    fmt= '.'
)
ax_y.errorbar(
    ds, rds.fit_params.sel(name=quad_names, parameter='scaled_angle', result='value', plane='y') * a_scale, 
    yerr=rds.fit_params.sel(name=quad_names, parameter='scaled_angle', result='error', plane='y') * err_scale, 
    fmt= '.'
)
ax_y.set_xlabel('s [m]')
ax_x.set_ylabel('x [$\mu rad$]')
ax_y.set_ylabel('y [$\mu$ rad]')


In [ ]:
import bact2_bessyii.magnets
importlib.reload(bact2_bessyii.magnets)

In [ ]:
import bact_analysis_bessyii.bba.calc
importlib.reload(bact_analysis_bessyii.bba.calc)

In [ ]:
calib_quad = bact_analysis_bessyii.bba.calc.load_calib_data()
calib_quad

In [ ]:
offsets = bact_analysis_bessyii.bba.calc.angles_to_offset_all(rds, names=quad_names)
offsets;

In [ ]:
rds.n

In [ ]:
name

In [ ]:
offsets.sel(name=name, result="value")*1e3*2

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=[12, 9], sharex=True)
ax_x, ax_y = axes
a_scale = 1e6
ax_x.errorbar(
    ds,
    offsets.sel(name=quad_names, result="value", plane="x") * a_scale,
    yerr=offsets.sel(name=quad_names, result="error", plane="x") * a_scale,
    fmt=".",
)
ax_y.errorbar(
    ds,
    offsets.sel(name=quad_names, result="value", plane="y") * a_scale,
    yerr=offsets.sel(name=quad_names, result="error", plane="y") * a_scale,
    fmt=".",
)
ax_y.set_xlabel("s [m]")
ax_x.set_ylabel("x [$\mu$m]")
ax_y.set_ylabel("y [$\mu$m]")

## Comparison to classic BBA

In [ ]:
import os.path
import pandas as pd
import h5py

In [ ]:
ls BBA_Classic

In [ ]:
t_dir = 'BBA_Classic'

In [ ]:
df_classic  = pd.read_hdf(os.path.join(t_dir, "20210818_quadrupoleOffsetsOldSchoolAnalysis.hdf")).set_index("names")

In [ ]:
ds_classic = xr.Dataset.from_dataframe(df_classic)
ds_classic = ds_classic.assign_coords(names=[name.strip() for name in ds_classic.names.values])
ds_classic

In [ ]:
ds_classic.sel(names="Q5M2T5R")*1e3

In [ ]:
ds;

In [ ]:
quad_names = offsets.coords['name'].values
quad_names_lc = [name.lower() for name in quad_names]
quad_names;

In [ ]:
set(ds_classic.names.values).difference(quad_names)

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=[12, 9], sharex=True)
ax_x, ax_y = axes
a_scale = 1e6
classic_scale = 1
ax_x.errorbar(
    ds.sel(pos=quad_names_lc),
    offsets.sel(name=quad_names, result="value", plane="x") * a_scale,
    yerr=offsets.sel(name=quad_names, result="error", plane="x") * a_scale,
    fmt=".",
)
ax_x.plot(ds_classic.s.sel(names=quad_names), ds_classic.hor_offset.sel(names=quad_names) * a_scale * classic_scale, '+')
ax_y.errorbar(
    ds,
    offsets.sel(name=quad_names, result="value", plane="y") * a_scale,
    yerr=offsets.sel(name=quad_names, result="error", plane="y") * a_scale,
    fmt=".",
)
ax_y.plot(ds_classic.s.sel(names=quad_names), ds_classic.ver_offset.sel(names=quad_names) * a_scale * classic_scale * -1, '+')
ax_y.set_xlabel("s [m]")
ax_x.set_ylabel("x [$\mu$m]")
ax_y.set_ylabel("y [$\mu$m]")

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=[12, 9], sharex=True)
ax_x, ax_y = axes
a_scale = 1e3
classic_scale = 1e3 * 1
ax_x.errorbar(
    ds,
    offsets.sel(name=quad_names, result="value", plane="x") - ds_classic.hor_offset.sel(name=quad_names),
    #yerr=offsets.sel(name=quad_names, result="error", plane="x") * a_scale,
    fmt=".",
)
ax_y.errorbar(
    ds,
    offsets.sel(name=quad_names, result="value", plane="y") - ds_classic.ver_offset.sel(name=quad_names),
    # yerr=offsets.sel(name=quad_names, result="error", plane="y") * a_scale,
    fmt=".",
)
ax_y.set_xlabel("s [m]")
ax_x.set_ylabel("x [$\mu$m]")
ax_y.set_ylabel("y [$\mu$m]")

In [ ]:
## Hacks below

In [ ]:
 df = bact2_bessyii.magnets.quadrupole_calbration_factors_mongodb()

In [ ]:
df.loc['Q5M2T5R']

In [ ]:
brho = 1.2 *4.23

In [ ]:
df.loc['Q5M2T5R'].hw2phys  * brho

In [ ]:
import epics

In [ ]:
pv = epics.PV(df.loc['Q5M2T5R'].Setpoint)
c_val = pv.get()
c_val, c_val * df.loc['Q5M2T5R'].hw2phys  * brho * 50e-3